<a href="https://colab.research.google.com/github/Ibrahim-Ezmazy/python-Projects/blob/main/VGG16_CoffeeDiseases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.metrics import confusion_matrix, classification_report


In [1]:
# Mount Google Drive if needed
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Define image data generators
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Path to your dataset folders
train_data_dir = '/content/drive/MyDrive/training'
val_data_dir = '/content/drive/MyDrive/validation'


In [4]:
# Define constants
batch_size = 20
num_classes = 2
epochs = 10

# Create image generators
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(150, 150),  # VGG16 input size
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
        val_data_dir,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')

# Load VGG16 model
base_model = VGG16(weights='imagenet', include_top=False)

# Add custom layers on top of VGG16
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


Found 419 images belonging to 2 classes.
Found 111 images belonging to 2 classes.
58889256/58889256 [==============================] - 0s 0us/step


In [5]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)


Epoch 1/10
20/20 [==============================] - 271s 13s/step - loss: 0.6283 - accuracy: 0.6767 - val_loss: 0.5663 - val_accuracy: 0.7900
Epoch 2/10
20/20 [==============================] - 182s 9s/step - loss: 0.5288 - accuracy: 0.8596 - val_loss: 0.4957 - val_accuracy: 0.8600
Epoch 3/10
20/20 [==============================] - 225s 11s/step - loss: 0.4650 - accuracy: 0.8872 - val_loss: 0.4373 - val_accuracy: 0.8500
Epoch 4/10
20/20 [==============================] - 184s 9s/step - loss: 0.4147 - accuracy: 0.9023 - val_loss: 0.3988 - val_accuracy: 0.8800
Epoch 5/10
20/20 [==============================] - 179s 9s/step - loss: 0.3800 - accuracy: 0.9098 - val_loss: 0.3827 - val_accuracy: 0.8700
Epoch 6/10
20/20 [==============================] - 188s 9s/step - loss: 0.3472 - accuracy: 0.9173 - val_loss: 0.3464 - val_accuracy: 0.8700
Epoch 7/10
20/20 [==============================] - 181s 9s/step - loss: 0.3230 - accuracy: 0.9223 - val_loss: 0.3055 - val_accuracy: 0.8900
Epoch 8/10


In [6]:
# Evaluate the model
validation_generator.reset()
y_pred = model.predict(validation_generator)
y_true = validation_generator.classes


6/6 [==============================] - 40s 6s/step


In [9]:
import numpy as np
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
evaluation = model.evaluate(validation_generator)
# Evaluate the model
validation_generator.reset()  # Reset the generator to avoid shuffling issues
y_pred = model.predict(validation_generator)
y_true = validation_generator.classes

# Convert predictions to class labels
y_pred_classes = np.argmax(y_pred, axis=1)

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print("Confusion Matrix:")
print(conf_matrix)

# Calculate precision, recall, and F-score
precision = precision_score(y_true, y_pred_classes)
recall = recall_score(y_true, y_pred_classes)
f_score = f1_score(y_true, y_pred_classes)

print("Precision:", precision)
print("Recall:", recall)
print("F-score:", f_score)


6/6 [==============================] - 41s 6s/step
Confusion Matrix:
[[24 24]
 [29 34]]
Precision: 0.5862068965517241
Recall: 0.5396825396825397
F-score: 0.5619834710743802
